In [ ]:
print ("============================================================")
print (" Course Work                              Robiul Islam      ")
print ("============================================================")


import pandas as pd 
from sklearn.utils import shuffle
from itertools import combinations
import bisect
import math

data = pd.read_csv("datafull.csv",sep=';') 
gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
data['Genaration']=gen
# Remove Null value
data.dropna(inplace = True)
data = shuffle(data, random_state=7)
data.head()
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def shapley_calc(N,characteristic_function,n):
    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                try:
                    cmod = len(j)
                    Cui = j[:]
                    l = N.index(j)
                    k = N.index(Cui)
                    temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                               float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                    shapley += temp
                except:
                    temp = float(N[0][0])
                    shapley += temp

        shapley_values.append(shapley)
    return shapley_values 

result =[]
for row in range(data.shape[0]):
    PS = power_set(data.iloc[row])
    Chan_fun =[]
    for i in PS:
        Chan_fun.append(max(i))
    Value = shapley_calc(PS,Chan_fun,5)
    result.append(max(Value))
    
data['result'] = result
print(data.head())

# gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
# print(gen.head())

# This code is 

In [15]:

from keras.models import Sequential
from keras.layers import Dense
import numpy
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
numpy.random.seed(7)
batch_size = 5
# load pima indians dataset
# split into input (X) and output (Y) variables
X = data[['work','separation','partner','marriage','children','result']].values
y = data['Genaration'].values
data.loc[data['Generation'] <= 3, 'Generation'] = 0
data.loc[data['Generation'] >  3, 'Generation'] = 1
y = data['Genaration'].values

# create model

score =[]
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    model = Sequential()
    model.add(Dense(12, input_dim=6, activation='relu'))
    model.add(Dense(6, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
# Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
    model.fit(X_train, y_train, epochs=100,batch_size=batch_size)
# evaluate the model
    s = model.evaluate(X_test, y_test)
    score.append(s)
    print("\n%s: %.2f%%" % (model.metrics_names[1], s[1]*100))

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
247/247 [==============================] - 0s 2ms/step - loss: -48.5804 - acc: 0.0162
Epoch 2/100
247/247 [==============================] - 0s 316us/step - loss: -50.5352 - acc: 0.0162
Epoch 3/100
247/247 [==============================] - 0s 292us/step - loss: -50.5380 - acc: 0.0162
Epoch 4/100
247/247 [==============================] - 0s 320us/step - loss: -50.5380 - acc: 0.0162
Epoch 5/100
247/247 [==============================] - 0s 344us/step - loss: -50.5380 - acc: 0.0162
Epoch 6/100
247/247 [==============================] - 0s 345us/step - loss: -50.5380 - acc: 0.0162
Epoch 7/100
247/247 [==============================] - 0s 311us/step - loss: -50.5380 - acc: 0.0162
Epoch 8/100
247/247 [==============================] - 0s 302us/step - loss: -50.5380 - acc: 0.0162
Epoch 9/100
247/247 [==============================] - 0s 351us/step - loss: -50.5380 - acc: 0.0162
Epoch 10/100
247/247 [==============================] 

247/247 [==============================] - 0s 372us/step - loss: -50.5380 - acc: 0.0162
Epoch 80/100
247/247 [==============================] - 0s 289us/step - loss: -50.5380 - acc: 0.0162
Epoch 81/100
247/247 [==============================] - 0s 266us/step - loss: -50.5380 - acc: 0.0162
Epoch 82/100
247/247 [==============================] - 0s 607us/step - loss: -50.5380 - acc: 0.0162
Epoch 83/100
247/247 [==============================] - 0s 567us/step - loss: -50.5380 - acc: 0.0162
Epoch 84/100
247/247 [==============================] - 0s 307us/step - loss: -50.5380 - acc: 0.0162
Epoch 85/100
247/247 [==============================] - 0s 288us/step - loss: -50.5380 - acc: 0.0162
Epoch 86/100
247/247 [==============================] - 0s 273us/step - loss: -50.5380 - acc: 0.0162
Epoch 87/100
247/247 [==============================] - 0s 285us/step - loss: -50.5380 - acc: 0.0162
Epoch 88/100
247/247 [==============================] - 0s 563us/step - loss: -50.5380 - acc: 0.0162
Epo

247/247 [==============================] - 0s 812us/step - loss: -50.5380 - acc: 0.0162
Epoch 60/100
247/247 [==============================] - 0s 317us/step - loss: -50.5380 - acc: 0.0162
Epoch 61/100
247/247 [==============================] - 0s 297us/step - loss: -50.5380 - acc: 0.0162
Epoch 62/100
247/247 [==============================] - 0s 285us/step - loss: -50.5380 - acc: 0.0162
Epoch 63/100
247/247 [==============================] - 0s 282us/step - loss: -50.5380 - acc: 0.0162
Epoch 64/100
247/247 [==============================] - 0s 567us/step - loss: -50.5380 - acc: 0.0162
Epoch 65/100
247/247 [==============================] - 0s 308us/step - loss: -50.5380 - acc: 0.0162
Epoch 66/100
247/247 [==============================] - 0s 278us/step - loss: -50.5380 - acc: 0.0162
Epoch 67/100
247/247 [==============================] - 0s 283us/step - loss: -50.5380 - acc: 0.0162
Epoch 68/100
247/247 [==============================] - 0s 315us/step - loss: -50.5380 - acc: 0.0162
Epo

247/247 [==============================] - 0s 307us/step - loss: -5.7732 - acc: 0.0162
Epoch 41/100
247/247 [==============================] - 0s 289us/step - loss: -5.9299 - acc: 0.0162
Epoch 42/100
247/247 [==============================] - 0s 284us/step - loss: -6.0860 - acc: 0.0162
Epoch 43/100
247/247 [==============================] - 0s 285us/step - loss: -6.2421 - acc: 0.0162
Epoch 44/100
247/247 [==============================] - 0s 291us/step - loss: -6.3974 - acc: 0.0162
Epoch 45/100
247/247 [==============================] - 0s 287us/step - loss: -6.5526 - acc: 0.0162
Epoch 46/100
247/247 [==============================] - 0s 284us/step - loss: -6.7071 - acc: 0.0162
Epoch 47/100
247/247 [==============================] - 0s 285us/step - loss: -6.8624 - acc: 0.0162
Epoch 48/100
247/247 [==============================] - 0s 299us/step - loss: -7.0165 - acc: 0.0162
Epoch 49/100
247/247 [==============================] - 0s 287us/step - loss: -7.1703 - acc: 0.0162
Epoch 50/100


Epoch 21/100
247/247 [==============================] - 0s 281us/step - loss: -50.5380 - acc: 0.0162
Epoch 22/100
247/247 [==============================] - 0s 285us/step - loss: -50.5380 - acc: 0.0162
Epoch 23/100
247/247 [==============================] - 0s 282us/step - loss: -50.5380 - acc: 0.0162
Epoch 24/100
247/247 [==============================] - 0s 280us/step - loss: -50.5380 - acc: 0.0162
Epoch 25/100
247/247 [==============================] - 0s 287us/step - loss: -50.5380 - acc: 0.0162
Epoch 26/100
247/247 [==============================] - 0s 287us/step - loss: -50.5380 - acc: 0.0162
Epoch 27/100
247/247 [==============================] - 0s 286us/step - loss: -50.5380 - acc: 0.0162
Epoch 28/100
247/247 [==============================] - 0s 279us/step - loss: -50.5380 - acc: 0.0162
Epoch 29/100
247/247 [==============================] - 0s 279us/step - loss: -50.5380 - acc: 0.0162
Epoch 30/100
247/247 [==============================] - 0s 271us/step - loss: -50.5380 - ac

...

247/247 [==============================] - 0s 349us/step - loss: -50.5380 - acc: 0.0162
Epoch 43/100
247/247 [==============================] - 0s 413us/step - loss: -50.5380 - acc: 0.0162
Epoch 44/100
247/247 [==============================] - 0s 431us/step - loss: -50.5380 - acc: 0.0162
Epoch 45/100
247/247 [==============================] - 0s 341us/step - loss: -50.5380 - acc: 0.0162
Epoch 46/100
247/247 [==============================] - 0s 340us/step - loss: -50.5380 - acc: 0.0162
Epoch 47/100
247/247 [==============================] - 0s 337us/step - loss: -50.5380 - acc: 0.0162
Epoch 48/100
247/247 [==============================] - 0s 403us/step - loss: -50.5380 - acc: 0.0162
Epoch 49/100
247/247 [==============================] - 0s 330us/step - loss: -50.5380 - acc: 0.0162
Epoch 50/100
247/247 [==============================] - 0s 339us/step - loss: -50.5380 - acc: 0.0162
Epoch 51/100
247/247 [==============================] - 0s 294us/step - loss: -50.5380 - acc: 0.0162
Epo

247/247 [==============================] - 0s 304us/step - loss: -50.5380 - acc: 0.0162
Epoch 23/100
247/247 [==============================] - 0s 291us/step - loss: -50.5380 - acc: 0.0162
Epoch 24/100
247/247 [==============================] - 0s 288us/step - loss: -50.5380 - acc: 0.0162
Epoch 25/100
247/247 [==============================] - 0s 292us/step - loss: -50.5380 - acc: 0.0162
Epoch 26/100
247/247 [==============================] - 0s 289us/step - loss: -50.5380 - acc: 0.0162
Epoch 27/100
247/247 [==============================] - 0s 287us/step - loss: -50.5380 - acc: 0.0162
Epoch 28/100
247/247 [==============================] - 0s 290us/step - loss: -50.5380 - acc: 0.0162
Epoch 29/100
247/247 [==============================] - 0s 284us/step - loss: -50.5380 - acc: 0.0162
Epoch 30/100
247/247 [==============================] - 0s 285us/step - loss: -50.5380 - acc: 0.0162
Epoch 31/100
247/247 [==============================] - 0s 294us/step - loss: -50.5380 - acc: 0.0162
Epo

247/247 [==============================] - 0s 304us/step - loss: -50.5380 - acc: 0.0162
Epoch 3/100
247/247 [==============================] - 0s 296us/step - loss: -50.5380 - acc: 0.0162
Epoch 4/100
247/247 [==============================] - 0s 304us/step - loss: -50.5380 - acc: 0.0162
Epoch 5/100
247/247 [==============================] - 0s 301us/step - loss: -50.5380 - acc: 0.0162
Epoch 6/100
247/247 [==============================] - 0s 295us/step - loss: -50.5380 - acc: 0.0162
Epoch 7/100
247/247 [==============================] - 0s 274us/step - loss: -50.5380 - acc: 0.0162
Epoch 8/100
247/247 [==============================] - 0s 282us/step - loss: -50.5380 - acc: 0.0162
Epoch 9/100
247/247 [==============================] - 0s 295us/step - loss: -50.5380 - acc: 0.0162
Epoch 10/100
247/247 [==============================] - 0s 293us/step - loss: -50.5380 - acc: 0.0162
Epoch 11/100
247/247 [==============================] - 0s 290us/step - loss: -50.5380 - acc: 0.0162
Epoch 12/1

247/247 [==============================] - 0s 297us/step - loss: -50.5380 - acc: 0.0162
Epoch 84/100
247/247 [==============================] - 0s 298us/step - loss: -50.5380 - acc: 0.0162
Epoch 85/100
247/247 [==============================] - 0s 297us/step - loss: -50.5380 - acc: 0.0162
Epoch 86/100
247/247 [==============================] - 0s 297us/step - loss: -50.5380 - acc: 0.0162
Epoch 87/100
247/247 [==============================] - 0s 295us/step - loss: -50.5380 - acc: 0.0162
Epoch 88/100
247/247 [==============================] - 0s 296us/step - loss: -50.5380 - acc: 0.0162
Epoch 89/100
247/247 [==============================] - 0s 296us/step - loss: -50.5380 - acc: 0.0162
Epoch 90/100
247/247 [==============================] - 0s 303us/step - loss: -50.5380 - acc: 0.0162
Epoch 91/100
247/247 [==============================] - 0s 291us/step - loss: -50.5380 - acc: 0.0162
Epoch 92/100
247/247 [==============================] - 0s 280us/step - loss: -50.5380 - acc: 0.0162
Epo

247/247 [==============================] - 0s 292us/step - loss: -50.5380 - acc: 0.0162
Epoch 64/100
247/247 [==============================] - 0s 295us/step - loss: -50.5380 - acc: 0.0162
Epoch 65/100
247/247 [==============================] - 0s 293us/step - loss: -50.5380 - acc: 0.0162
Epoch 66/100
247/247 [==============================] - 0s 291us/step - loss: -50.5380 - acc: 0.0162
Epoch 67/100
247/247 [==============================] - 0s 306us/step - loss: -50.5380 - acc: 0.0162
Epoch 68/100
247/247 [==============================] - 0s 298us/step - loss: -50.5380 - acc: 0.0162
Epoch 69/100
247/247 [==============================] - 0s 300us/step - loss: -50.5380 - acc: 0.0162
Epoch 70/100
247/247 [==============================] - 0s 299us/step - loss: -50.5380 - acc: 0.0162
Epoch 71/100
247/247 [==============================] - 0s 297us/step - loss: -50.5380 - acc: 0.0162
Epoch 72/100
247/247 [==============================] - 0s 302us/step - loss: -50.5380 - acc: 0.0162
Epo

In [23]:
type(score)
score= np.array(score)
print(np.mean(score[:,1]*100))

4.838709677419354


In [21]:
import shap
import numpy as np

# select a set of background examples to take an expectation over
background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

# explain predictions of the model on four images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(X_train)
shap_values = np.array(shap_values)


In [22]:
shap_values[0]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [1]:
print('hello world')

hello world


In [4]:
import lutorpy as lua
x =  torch.Tensor([[1, 2, 3], [1, 2, 3]]).view(-1, 2)
y =  torch.Tensor([[2, 1]]).view(2, -1)

ModuleNotFoundError: No module named 'lutorpy'

In [13]:
data.loc[data['Generation'] <= 3, 'Generation'] = 0
data.loc[data['Generation'] >  3, 'Generation'] = 1

In [14]:
data['Generation']

283     0
1329    0
2051    0
450     0
1135    0
1398    0
237     0
702     0
1037    0
913     0
505     0
2286    0
1679    0
1971    0
526     0
659     0
1247    0
1221    0
841     0
1106    0
20      0
1677    0
675     0
1601    0
463     0
1579    0
1286    0
603     0
155     0
1891    0
       ..
1585    0
1013    0
1953    0
306     0
1605    0
1561    0
1936    0
827     0
1483    0
1130    0
1575    0
353     0
2017    0
35      0
1933    0
539     0
0       0
481     0
1285    0
1730    0
342     0
513     0
207     0
1526    0
733     0
1674    0
476     0
214     0
1587    0
1383    0
Name: Generation, Length: 309, dtype: int64

In [20]:
data.loc[data['Generation'] <= 3, 'Generation'] = 0
data.loc[data['Generation'] >  3, 'Generation'] = 1
y = data['Genaration'].values
print(y)

[6 4 5 4 2 0 5 5 6 5 0 4 4 4 4 1 6 6 4 3 6 2 6 5 6 6 5 6 4 6 3 6 5 2 6 5 6
 0 4 3 4 4 4 5 4 5 0 0 3 5 4 3 5 3 4 1 6 6 2 3 3 5 6 1 3 4 4 6 4 5 4 4 4 5
 4 2 1 5 6 6 6 4 5 3 5 6 6 4 4 5 5 5 5 4 6 6 4 3 4 4 6 6 5 3 5 6 3 3 2 0 2
 5 3 3 4 4 0 4 3 4 2 1 0 5 5 3 6 0 5 6 3 3 6 5 4 6 5 2 5 5 6 5 4 6 2 2 5 4
 5 5 0 3 6 3 5 0 4 5 5 0 3 4 5 4 5 5 6 4 5 6 2 4 6 5 5 1 0 5 4 3 5 5 5 3 3
 6 5 0 6 4 1 5 5 6 6 6 6 6 4 6 2 6 4 4 4 3 5 0 4 2 3 6 3 6 6 6 6 4 6 3 4 4
 5 2 2 6 5 6 2 2 5 3 5 6 2 6 5 5 6 4 4 2 5 6 3 3 6 5 6 4 5 6 5 6 4 5 2 6 5
 0 3 6 6 5 5 2 6 0 4 3 6 3 4 2 4 4 5 5 5 0 4 4 5 2 4 5 6 6 5 2 0 3 2 5 4 6
 3 2 5 4 5 5 3 2 2 5 5 0 5]


In [25]:
X,y

(array([[ 1.76041518e+00, -3.91816240e-01,  5.56073510e-01,
         -2.51630386e+00, -1.16400797e+00,  6.36583409e-01,
         -8.06184817e-01, -1.83400197e-01, -1.78791289e+00,
         -1.15498263e+00,  5.10937774e-01,  1.44287693e+00,
          1.81038744e-01,  1.47016034e+00,  9.66539250e-01,
          3.82715174e-01,  1.62091229e+00,  1.89293198e-01,
          1.07343294e-01,  5.13505480e-01, -1.51045638e+00,
         -2.75293863e-02,  1.28376990e-01, -7.99547491e-01,
         -2.22414029e-01,  5.48129585e-01, -1.47656266e+00,
          1.74643509e+00, -6.74100249e-02, -1.56573815e+00,
          3.95860270e+00, -5.39681562e-01,  1.71066184e+00,
         -6.04602969e-01,  1.69380911e+00, -2.91594596e-01,
          1.01120706e+00,  5.57439453e-01, -1.11343610e+00,
         -7.31530982e-01, -1.63744959e+00, -2.51674208e-01,
          7.13049050e-01, -4.24492791e-01,  1.30620607e+00,
         -3.09121319e-01,  9.86352180e-01,  1.85300949e+00,
         -2.58982853e-01,  3.69190470e-0

In [26]:
print("Hello")

Hello
